In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from preprocessing import clsf_boz_cd, clsf_rrnr_dvcd, clsf_dmfr_dvcd, clsf_crd_grd, get_cf, clsf_pdgr_cd, get_disc_factor

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_일반')

In [3]:
# 데이터 불러오기
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
일반_원수_미경과보험료 = pd.read_excel(FILE_PATH / '일반_원수_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_미경과보험료 = pd.read_excel(FILE_PATH / '일반_출재_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / '재보험자_국내신용등급.xlsx', dtype={'재보험사코드': str}) \
    .rename(columns = {'재보험사코드': 'T02_RN_RINSC_CD', '국내신용등급': 'CRD_GRD'})
일반_보험금진전추이 = pd.read_excel(FILE_PATH / '일반_보험금진전추이.xlsx')
할인율 = pd.read_excel(FILE_PATH / '할인율.xlsx')

In [14]:
# 데이터 전처리
일반_원수_미경과보험료['BOZ_CD'] = clsf_boz_cd(일반_원수_미경과보험료, 일반_상품정보)
일반_원수_미경과보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_미경과보험료, '원수')
일반_원수_미경과보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_미경과보험료)
일반_원수_미경과보험료['PDGR_CD'] = clsf_pdgr_cd(일반_원수_미경과보험료, 일반_상품정보)

일반_출재_미경과보험료['BOZ_CD'] = clsf_boz_cd(일반_출재_미경과보험료, 일반_상품정보)
일반_출재_미경과보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_미경과보험료, '출재')
일반_출재_미경과보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_미경과보험료)
일반_출재_미경과보험료['CRD_GRD'] = clsf_crd_grd(일반_출재_미경과보험료, 재보험자_국내신용등급)
일반_출재_미경과보험료['PDGR_CD'] = clsf_pdgr_cd(일반_출재_미경과보험료, 일반_상품정보)

Signature: get_cf(cf: pandas.core.frame.DataFrame, pdgr_cd: str, cf_type: str) -> Tuple[pandas.core.series.Series, pandas.core.series.Series]
Docstring:
보험금/보험료 현금흐름비중 계산

Args:
    cf (pd.DataFrame): 보험금 진전추이
    pdgr_cd (str): 상품군코드
    cf_type (str): "보험금" 또는 "보험료"

Raises:
    Exception: [description]
    Exception: [description]
    Exception: [description]

Returns:
    Tuple[pd.Series, pd.Series]: (지급시점, 현금흐름비중)

Example:
    일반_보험금진전추이 = pd.read_excel('data/현행추정부채_일반/일반_보험금진전추이.xlsx')
    pdgr_cd = '26'
    cf_type = '보험료'
    cf_t, cf = get_cf(일반_보험금진전추이.query('PDGR_CD == @pdgr_cd'), pdgr_cd, cf_type)
File:      c:\users\11700205\workspace\96. 깃허브\qis4\preprocessing.py
Type:      function


In [8]:
# 데이터 집계
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])['LTPD_URND_PRM'].sum().reset_index()
일반_원수_미경과보험료_집계.head()

,RRNR_DVCD,DMFR_DVCD,BOZ_CD,LTPD_URND_PRM
0,01,01,A001,23420916171
1,01,01,A002,27146263273
2,01,01,A003,96625029260
3,01,01,A004,22191701960
4,01,01,A005,5095840261


In [ ]:
# 데이터 내보내기